# 03_ai_agent_analysis_part1_fixed — Config & API 拡張（統合版）
このノートブックは、**元コードを温存**したまま、以下のセルを**追記**して機能拡張します：
- セル0: Config読み込みとマージ (`load_configuration`)
- セル3(パッチ): DB/LLM設定のConfig反映 + GPU自動検出 + vLLM→Ollamaフォールバック
- セルX: 偽陰性分析の閾値をConfigから取得（低確率領域分析は必ず実行）
- 最終セル: Controller API (`agent_minimal`)
※ 元セルは変更・削除せず、**実行順序で上書き適用**します（副作用回避）。


In [1]:
import matplotlib as mpl
from matplotlib import font_manager as fm

# どのフォントが選ばれるかは rcParams で決まるので、最優先に Noto を置く
mpl.rcParams["font.family"] = "sans-serif"
mpl.rcParams["font.sans-serif"] = ["Noto Sans CJK JP", "DejaVu Sans"]
mpl.rcParams["axes.unicode_minus"] = False

# 念のため「本当に見えてるか」も確認
print("Using candidate:", mpl.rcParams["font.sans-serif"][0])
print("Resolved path:", fm.findfont("Noto Sans CJK JP"))


Using candidate: Noto Sans CJK JP
Resolved path: /usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc


In [2]:
# === Cell 0 (02以降 共通): レジストリから解決して paths を読む ===
import run_id_registry as runreg
rid = runreg.bootstrap()  # env→ファイル(artifacts/_current/run_id.txt)→Part3→latest→新規 の順で解決

import importlib
import _compat.paths as paths
importlib.reload(paths)
importlib.reload(paths)
print("[NX] RUN_ID =", rid, "| paths.RUN_ID =", paths.RUN_ID)


[NX] RUN_ID = 2026-02-02_220431 | paths.RUN_ID = 2026-02-02_220431


In [3]:
# === セル0: Config読み込みとマージ (厳格版) ===
from __future__ import annotations
from typing import Optional, Dict, Any, Tuple
import os, json
from pathlib import Path
try:
    import yaml  # optional
except Exception:
    yaml = None

def _deep_update(base: dict, override: dict) -> dict:
    base = dict(base or {})
    for k, v in (override or {}).items():
        if isinstance(v, dict) and isinstance(base.get(k), dict):
            base[k] = _deep_update(base[k], v)
        else:
            base[k] = v
    return base

def load_configuration(
    config_path: Optional[str] = None,
    cfg_override: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    
    # 1) システムの動作フラグのみデフォルトを持つ (環境依存値は一切ハードコードしない)
    defaults = {
        "system": {
            "cert_only_mode": False,
            "enable_brand_keywords": True,
            "gpu_auto_detect": True,
            "development_mode": False,
            "seed": 42,
        },
        "llm": {
            "provider": "vllm", 
            "fallback_enabled": False, # フォールバックは無効化
            "temperature": 0.1,
            "max_tokens": 2000,
            # URLやモデル名はConfigファイル必須とする
        },
        "db": {
            "timeout_s": 30,
            "read_only": True,
            # DB接続情報もConfigファイル必須
        },
        "analysis": {
            "fn_threshold_low": 0.2,
            "fn_threshold_high": 0.7,
            "enable_detailed_stats": True,
        },
    }
    cfg = dict(defaults)

    # 2) Configファイルの探索 (指定パス -> カレント -> _compat -> 親)
    search_paths = []
    if config_path:
        search_paths.append(config_path)
    
    # 自動探索パス
    search_paths.extend(["config.json", "_compat/config.json", "../config.json"])
    
    loaded_path = None
    file_cfg = {}

    for p in search_paths:
        if p and os.path.exists(p):
            try:
                text = Path(p).read_text(encoding="utf-8")
                if p.endswith((".yml", ".yaml")) and yaml is not None:
                    file_cfg = yaml.safe_load(text) or {}
                else:
                    file_cfg = json.loads(text)
                
                loaded_path = p
                print(f"📖 設定ファイルを読み込みました: {loaded_path}")
                break
            except Exception as e:
                print(f"⚠️ ファイル {p} の読み込みに失敗しました: {e}")
                # 読み込みエラーは致命的として扱う場合はここでraiseしても良いが、
                # 次の候補を探すためにcontinueする
                continue

    # ★変更点: 設定ファイルが見つからなければ即エラー (フォールバックなし)
    if not loaded_path:
        raise FileNotFoundError(
            "❌ 必須の設定ファイル (config.json) が見つかりません。\n"
            "   実行ディレクトリ、または _compat/ 配下に config.json を配置してください。"
        )

    # 設定のマージ
    cfg = _deep_update(cfg, file_cfg)

    # 3) 環境変数 (Env vars) での上書きは利便性のため残す
    env_map = {
        "DEV_MODE": ("system", "development_mode"),
        "LLM_TYPE": ("llm", "provider"),
        "VLLM_BASE_URL": ("llm", "vllm_base_url"),
        "PGDATABASE": ("db", "dbname"),
        "PGUSER": ("db", "user"),
        "PGPASSWORD": ("db", "password"),
    }
    for env, path in env_map.items():
        if env in os.environ and os.environ[env] != "":
            val = os.environ[env]
            low = val.lower()
            if low in ("true","false"):
                val = (low == "true")
            node = cfg
            for key in path[:-1]:
                node = node.setdefault(key, {})
            node[path[-1]] = val

    # 4) 引数でのOverride
    if cfg_override:
        cfg = _deep_update(cfg, cfg_override)

    # 5) 必須項目のバリデーション
    # LLM設定チェック
    llm_conf = cfg.get("llm", {})
    provider = str(llm_conf.get("provider", "vllm")).lower()
    
    if provider == "vllm":
        if not llm_conf.get("vllm_base_url"):
            raise ValueError(f"❌ 設定ファイル {loaded_path} に 'llm.vllm_base_url' が定義されていません。")
    elif provider == "ollama":
        if not llm_conf.get("ollama_base_url"):
            raise ValueError(f"❌ 設定ファイル {loaded_path} に 'llm.ollama_base_url' が定義されていません。")

    # DB設定チェック
    db_conf = cfg.get("db", {})
    required_db_keys = ["dbname", "user", "host", "port"]
    missing_db_keys = [k for k in required_db_keys if not db_conf.get(k)]
    if missing_db_keys:
        raise ValueError(f"❌ 設定ファイル {loaded_path} にDB接続情報が不足しています: {missing_db_keys}")

    globals()["cfg"] = cfg
    print("✅ Config loaded successfully (provider={provider}, gpu_auto_detect={gpu})".format(
        provider=provider,
        gpu=cfg["system"]["gpu_auto_detect"],
    ))
    return cfg

In [4]:
import json
import pickle
import joblib  # 02_xgboost_training_evaluation.pyはjoblibで保存
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import os
import glob
import psycopg2
from psycopg2.extras import RealDictCursor
import warnings
import xgboost as xgb
import subprocess
import re

# === CHANGELOG ===
# 2025-12-28: artifacts/{session_id}/ 配下のフォルダ構成に合わせて入出力パスを全面的に更新（03_analysisに統一）

# === CONFIG LOADING (STRICT MODE) ===
CONFIG_PATH = Path("_compat/config.json")

if not CONFIG_PATH.exists():
    raise FileNotFoundError(f"❌ 必須設定ファイルが見つかりません: {CONFIG_PATH.absolute()}\n"
                            "   デフォルト設定は廃止されました。必ずconfig.jsonを作成してください。")

try:
    with open(CONFIG_PATH, 'r', encoding='utf-8') as f:
        APP_CONFIG = json.load(f)
    print(f"✅ 設定ファイルを読み込みました: {CONFIG_PATH}")
except json.JSONDecodeError as e:
    raise ValueError(f"❌ 設定ファイルのJSON形式が不正です: {e}")

# XGBoostのGPU警告を抑制
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')

# === 設定値の適用（キー不足時はKeyErrorで即停止） ===

# 1. System設定 (GPU検出フラグなど)
try:
    system_conf = APP_CONFIG["system"]
    gpu_auto_detect = system_conf["gpu_auto_detect"]
    DEVELOPMENT_MODE = system_conf["development_mode"]
except KeyError as e:
    raise KeyError(f"❌ config.jsonの 'system' セクションに必須キー {e} が不足しています。")

print(f"✅ 開発モード: {DEVELOPMENT_MODE}")

# ローカルGPUの確認（XGBoostモデル用）
GPU_AVAILABLE = True
if gpu_auto_detect:
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        if result.returncode == 0:
            print("✅ ローカルGPU検出: XGBoostモデルでGPU利用可能")
        else:
            print("⚠️ ローカルGPU未検出: XGBoostはCPU処理に切り替えます")
            GPU_AVAILABLE = False
    except:
        print("⚠️ nvidia-smi利用不可: XGBoostはCPU処理に切り替えます")
        GPU_AVAILABLE = False
else:
    print("ℹ️ GPU自動検出は無効化されています。")
    GPU_AVAILABLE = False


# 2. LLM設定
print("\n🤖 LLM設定:")
try:
    llm_conf = APP_CONFIG["llm"]
    # キー名のマッピング修正: "type" -> "provider"
    LLM_TYPE = llm_conf["provider"]
    VLLM_BASE_URL = llm_conf["vllm_base_url"]
    VLLM_MODEL = llm_conf["vllm_model"]
    OLLAMA_MODEL = llm_conf["ollama_model"]
    # 念のためOllamaのURLも取得（使う場合）
    OLLAMA_BASE_URL = llm_conf.get("ollama_base_url")
except KeyError as e:
    raise KeyError(f"❌ config.jsonの 'llm' セクションに必須キー {e} が不足しています。")

print(f"  - LLM実行環境: {LLM_TYPE}")
print(f"  - LLMモデル: {VLLM_MODEL if LLM_TYPE == 'vllm' else OLLAMA_MODEL}")
if LLM_TYPE == 'vllm':
    print(f"  - Base URL: {VLLM_BASE_URL}")
    print(f"  - 注: LLMはリモートサーバーのGPUで実行されます")


# 3. データベース設定
try:
    # "database" ではなく "db" セクションを使用
    db_source = APP_CONFIG["db"]

    # 接続に必要なキーを抽出して DB_CONFIG を構築
    DB_CONFIG = {
        "dbname": db_source["dbname"],
        "user": db_source["user"],
        "password": db_source["password"],
        "host": db_source["host"],
        "port": db_source["port"]
    }
except KeyError as e:
    raise KeyError(f"❌ config.jsonの 'db' セクションに不備があります: {e}")

print("\n🔧 環境設定とデータ読み込み")
print("=" * 80)

# ===== Session ID の検出（artifacts/{session_id}/... 構成に合わせる）=====
session_id = None
artifacts_dir = Path("artifacts")
current_run_id_path = artifacts_dir / "_current" / "run_id.txt"

def _parse_session_id(name: str):
    m = re.match(r"^(\d{4})-(\d{2})-(\d{2})_(\d{6})$", name)
    if not m:
        return None
    try:
        return datetime.strptime(name, "%Y-%m-%d_%H%M%S")
    except ValueError:
        return None

def _has_required_models(run_dir: Path) -> bool:
    return (
        (run_dir / "models" / "xgboost_model.pkl").exists()
        and (run_dir / "models" / "scaler.pkl").exists()
        and (run_dir / "models" / "feature_order.json").exists()
        and (run_dir / "models" / "brand_keywords.json").exists()
    )

# 1) まず _current/run_id.txt を優先
if current_run_id_path.exists():
    rid = current_run_id_path.read_text(encoding="utf-8").strip()
    if rid:
        candidate_dir = artifacts_dir / rid
        if candidate_dir.exists() and candidate_dir.is_dir() and _has_required_models(candidate_dir):
            session_id = rid
            print(f"✅ artifacts/_current からセッションID検出: {session_id}")
        else:
            print(f"⚠️ artifacts/_current/run_id.txt は見つかったが、必要ファイルが揃っていません: {rid}")

# 2) フォールバック：artifacts/ 配下を走査して最新を拾う
if not session_id:
    candidates = []
    if artifacts_dir.exists():
        for d in artifacts_dir.iterdir():
            if d.is_dir():
                ts = _parse_session_id(d.name)
                if ts and _has_required_models(d):
                    candidates.append((ts, d.name))
    if candidates:
        candidates.sort(key=lambda x: x[0])
        session_id = candidates[-1][1]
        print(f"✅ artifacts から最新セッション検出: {session_id}")

if not session_id:
    raise ValueError("セッションが見つかりません。artifacts/_current/run_id.txt を確認するか、02を先に実行してください。")

# ===== ファイルパス定義（artifacts/{session_id}/...）=====
model_path = f"artifacts/{session_id}/models/xgboost_model.pkl"
scaler_path = f"artifacts/{session_id}/models/scaler.pkl"
feature_order_path = f"artifacts/{session_id}/models/feature_order.json"
brand_keywords_path = f"artifacts/{session_id}/models/brand_keywords.json"

print("  model_path :", model_path)
print("  scaler_path:", scaler_path)
print("  feature_order_path :", feature_order_path, ("(存在しません)" if not Path(feature_order_path).exists() else ""))
print("  brand_keywords_path:", brand_keywords_path, ("(存在しません)" if not Path(brand_keywords_path).exists() else ""))

# モデルの読み込み（joblib形式）
try:
    model = joblib.load(model_path)
    print(f"✅ XGBoostモデル読み込み完了: {model_path}")
except FileNotFoundError:
    print(f"❌ モデルファイルが見つかりません: {model_path}")
    print("   02_xgboost_training_evaluation.pyを実行してモデルを作成してください。")
    raise

# XGBoostモデルのGPU設定（ローカル実行、利用可能な場合）
if GPU_AVAILABLE:
    try:
        if hasattr(model, 'set_param'):
            model.set_param({'device': 'cuda:0'})
        elif hasattr(model, 'get_booster'):
            model.get_booster().set_param({'device': 'cuda:0'})
            if hasattr(model, 'set_params'):
                model.set_params(device='cuda:0', tree_method='hist')
        print(f"✅ XGBoostモデル: ローカルGPUを使用")
    except Exception as e:
        print(f"⚠️ GPU設定エラー: {e}")
        print("   XGBoostはCPU処理を継続します")

# スケーラーの読み込み（joblib形式）
try:
    scaler = joblib.load(scaler_path)
    print(f"✅ スケーラー読み込み完了: {scaler_path}")
except FileNotFoundError:
    print(f"❌ スケーラーファイルが見つかりません: {scaler_path}")
    print("   02_xgboost_training_evaluation.pyを実行してください。")
    raise

# 特徴量順序の読み込み（JSON形式）
try:
    with open(feature_order_path, 'r') as f:
        feature_order = json.load(f)
    print(f"✅ 特徴量順序読み込み完了: {len(feature_order)}個の特徴量")
except FileNotFoundError:
    print(f"❌ 特徴量順序ファイルが見つかりません: {feature_order_path}")
    print("   02_xgboost_training_evaluation.pyを実行してください。")
    raise

# 🔥 重要: 動的ブランドキーワードの読み込み（JSON形式）
try:
    with open(brand_keywords_path, 'r', encoding='utf-8') as f:
        brand_keywords = json.load(f)
    print(f"✅ ブランドキーワード読み込み完了: {len(brand_keywords)}個のブランド")
    print(f"   ブランド例: {brand_keywords[:5]}")
except FileNotFoundError:
    print(f"❌ ブランドキーワードファイルが見つかりません: {brand_keywords_path}")
    print("   02_xgboost_training_evaluation.pyを実行してブランドキーワードを生成してください。")
    print("   このファイルはGPT-4o miniで動的に生成される重要なデータです。")
    raise

# 偽陰性データの読み込み
false_negatives_df = None

# artifacts/{session_id}/results/ を優先
fn_paths = [
    f"artifacts/{session_id}/results/false_negatives_reconstructed.pkl",
    f"artifacts/{session_id}/results/false_negatives.pkl",
]

for fn_path in fn_paths:
    if os.path.exists(fn_path):
        try:
            # joblibで読み込み（02_xgboost_training_evaluation.pyの形式）
            fn_data = joblib.load(fn_path)

            # データ形式の判定
            if isinstance(fn_data, pd.DataFrame):
                false_negatives_df = fn_data
            elif isinstance(fn_data, dict):
                if 'analysis_df' in fn_data:
                    false_negatives_df = fn_data['analysis_df']
                elif 'domains' in fn_data and 'predictions' in fn_data:
                    # 旧形式の場合は再構築
                    false_negatives_df = pd.DataFrame({
                        'domain': fn_data['domains'],
                        'prediction_proba': fn_data['predictions'],
                        'source': fn_data.get('sources', ['unknown'] * len(fn_data['domains']))
                    })

            if false_negatives_df is not None:
                print(f"✅ 偽陰性データ読み込み完了: {fn_path}")
                break

        except Exception as e:
            print(f"⚠️ {fn_path}の読み込みに失敗: {e}")
            continue

if false_negatives_df is None:
    print("❌ 偽陰性データが見つかりません。")
    print("   02_xgboost_training_evaluation.pyを実行してデータを生成してください。")
    raise FileNotFoundError("偽陰性データファイルが見つかりません")

# 偽陰性データの統計表示
print(f"\n📊 偽陰性（FN）データの統計:")
print(f"  - 総FN数: {len(false_negatives_df):,}件")

if 'prediction_proba' in false_negatives_df.columns:
    print(f"  - 平均予測確率: {false_negatives_df['prediction_proba'].mean():.4f}")
    print(f"  - 中央値: {false_negatives_df['prediction_proba'].median():.4f}")
    print(f"  - 最小予測確率: {false_negatives_df['prediction_proba'].min():.4f}")
    print(f"  - 最大予測確率: {false_negatives_df['prediction_proba'].max():.4f}")

    # 🔥 低確率領域のFN分布（問題の核心）
    fn_low = false_negatives_df[false_negatives_df['prediction_proba'] < 0.2]
    fn_mid = false_negatives_df[(false_negatives_df['prediction_proba'] >= 0.2) &
                                 (false_negatives_df['prediction_proba'] < 0.4)]
    fn_high = false_negatives_df[false_negatives_df['prediction_proba'] >= 0.7]
    print(f"\n🎯 低確率領域のFN分布（問題の核心）:")
    print(f"  - ML確率 < 0.2: {len(fn_low):,}件 ({len(fn_low)/len(false_negatives_df)*100:.1f}%)")
    print(f"  - ML確率 0.2-0.4: {len(fn_mid):,}件 ({len(fn_mid)/len(false_negatives_df)*100:.1f}%)")
    print(f"  - ML確率 >= 0.7: {len(fn_high):,}件 ({len(fn_high)/len(false_negatives_df)*100:.1f}%)")

# ソース別の統計
if 'source' in false_negatives_df.columns:
    print(f"\nソース別内訳:")
    source_counts = false_negatives_df['source'].value_counts()
    for source, count in source_counts.items():
        print(f"  - {source}: {count:,}件 ({count/len(false_negatives_df)*100:.1f}%)")

# 出力ディレクトリの作成（03_analysis に統一）
output_dirs = {
    "analysis": f"artifacts/{session_id}/analysis/03_analysis/",
    "logs":    f"artifacts/{session_id}/logs/03_analysis/",
    "traces":  f"artifacts/{session_id}/traces/03_analysis/",
}
# 互換性のため（既存セルが output_dirs['results'] を参照しても崩れないように）
output_dirs["results"] = output_dirs["analysis"]

for dir_name, dir_path in output_dirs.items():
    os.makedirs(dir_path, exist_ok=True)
    print(f"✅ {dir_name}ディレクトリ作成: {dir_path}")

# タイムスタンプ
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# グローバル変数（後のセルから参照）
cert_full_info_map = {}  # セル02で作成される
fn_features_df = None    # セル02で作成される

print(f"\n✅ 初期設定完了")
print(f"  - セッションID: {session_id}")
print(f"  - LLM実行: {LLM_TYPE}")
print(f"  - LLMモデル: {VLLM_MODEL if LLM_TYPE == 'vllm' else OLLAMA_MODEL}")
print(f"  - XGBoost: ローカル（{'GPU' if GPU_AVAILABLE else 'CPU'}使用）")
print(f"  - タイムスタンプ: {timestamp}")
print(f"  - 偽陰性件数: {len(false_negatives_df)}")
print(f"  - ブランド数: {len(brand_keywords)}")
print("=" * 80)

# 変数名の統一（セル07, 08との互換性のため）
ai_session_id = session_id
globals()['ai_session_id'] = ai_session_id
globals()['LLM_TYPE'] = LLM_TYPE
globals()['output_dirs'] = output_dirs
globals()['DEVELOPMENT_MODE'] = DEVELOPMENT_MODE
globals()['brand_keywords'] = brand_keywords  # 🔥 重要: グローバル変数として設定

print(f"\n🔧 変数名の統一完了:")
print(f"  - ai_session_id: {ai_session_id}")
print(f"  - brand_keywords: {len(brand_keywords)}個のブランドを利用可能")
print(f"  - LLM_TYPE: {LLM_TYPE}")


✅ 設定ファイルを読み込みました: _compat/config.json
✅ 開発モード: False


✅ ローカルGPU検出: XGBoostモデルでGPU利用可能

🤖 LLM設定:
  - LLM実行環境: vllm
  - LLMモデル: JunHowie/Qwen3-4B-Thinking-2507-GPTQ-Int8
  - Base URL: http://localhost:8000/v1
  - 注: LLMはリモートサーバーのGPUで実行されます

🔧 環境設定とデータ読み込み
⚠️ artifacts/_current/run_id.txt は見つかったが、必要ファイルが揃っていません: 2026-02-02_220431
✅ artifacts から最新セッション検出: 2026-01-24_213326
  model_path : artifacts/2026-01-24_213326/models/xgboost_model.pkl
  scaler_path: artifacts/2026-01-24_213326/models/scaler.pkl
  feature_order_path : artifacts/2026-01-24_213326/models/feature_order.json 
  brand_keywords_path: artifacts/2026-01-24_213326/models/brand_keywords.json 
✅ XGBoostモデル読み込み完了: artifacts/2026-01-24_213326/models/xgboost_model.pkl
✅ XGBoostモデル: ローカルGPUを使用
✅ スケーラー読み込み完了: artifacts/2026-01-24_213326/models/scaler.pkl
✅ 特徴量順序読み込み完了: 42個の特徴量
✅ ブランドキーワード読み込み完了: 223個のブランド
   ブランド例: ['aeonbank', 'aeoncard', 'amazon', 'amex', 'apple']


✅ 偽陰性データ読み込み完了: artifacts/2026-01-24_213326/results/false_negatives_reconstructed.pkl

📊 偽陰性（FN）データの統計:
  - 総FN数: 15,670件
  - 平均予測確率: 0.1736
  - 中央値: 0.0579
  - 最小予測確率: 0.0036
  - 最大予測確率: 0.9610

🎯 低確率領域のFN分布（問題の核心）:
  - ML確率 < 0.2: 12,196件 (77.8%)
  - ML確率 0.2-0.4: 1,318件 (8.4%)
  - ML確率 >= 0.7: 1,310件 (8.4%)

ソース別内訳:
  - trusted: 12,931件 (82.5%)
  - jpcert: 1,240件 (7.9%)
  - certificates: 775件 (4.9%)
  - phishtank: 724件 (4.6%)
✅ analysisディレクトリ作成: artifacts/2026-01-24_213326/analysis/03_analysis/
✅ logsディレクトリ作成: artifacts/2026-01-24_213326/logs/03_analysis/
✅ tracesディレクトリ作成: artifacts/2026-01-24_213326/traces/03_analysis/
✅ resultsディレクトリ作成: artifacts/2026-01-24_213326/analysis/03_analysis/

✅ 初期設定完了
  - セッションID: 2026-01-24_213326
  - LLM実行: vllm
  - LLMモデル: JunHowie/Qwen3-4B-Thinking-2507-GPTQ-Int8
  - XGBoost: ローカル（GPU使用）
  - タイムスタンプ: 20260202_221835
  - 偽陰性件数: 15670
  - ブランド数: 223

🔧 変数名の統一完了:
  - ai_session_id: 2026-01-24_213326
  - brand_keywords: 223個のブランドを利用可能
  - LLM_TYPE: vllm

In [5]:
# === セル3(パッチ): DB/LLM設定をConfigから取得 & GPU制御 (厳格版: フォールバックなし) ===
from typing import Dict, Any, Optional
import subprocess

def _summarize_db(db: Dict[str, Any]) -> Dict[str, Any]:
    return {k: db.get(k) for k in ("dbname","user","host","port")}

def apply_config_to_runtime(cfg: Dict[str, Any]) -> Dict[str, Any]:
    # --- DB ---
    db = cfg.get("db", {})
    timeout_s = int(db.get("timeout_s", 30))
    DB_CONFIG = {
        "dbname": str(db.get("dbname")),
        "user": str(db.get("user")),
        "password": str(db.get("password","")),
        "host": str(db.get("host")),
        "port": str(db.get("port")),
    }
    globals()["DB_CONFIG"] = DB_CONFIG
    globals()["DB_CONNECT_TIMEOUT_S"] = timeout_s
    print("🔧 DB設定:", _summarize_db(DB_CONFIG), f"(timeout_s={timeout_s})")

    # --- GPU detect ---
    GPU_AVAILABLE = False
    if cfg.get("system", {}).get("gpu_auto_detect", True):
        try:
            r = subprocess.run(["nvidia-smi"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            GPU_AVAILABLE = (r.returncode == 0)
            print("✅ GPU検出: 利用可能" if GPU_AVAILABLE else "⚠️ GPU未検出: CPUで実行します")
        except Exception:
            print("⚠️ nvidia-smi 実行不可: CPUで実行します")
    else:
        print("ℹ️ GPU自動検出は無効化 (cfg.system.gpu_auto_detect=False)")
    globals()["GPU_AVAILABLE"] = GPU_AVAILABLE

    # --- LLM ---
    try:
        from openai import OpenAI  # OpenAI SDK 互換 (vLLM/Ollama)
    except Exception:
        OpenAI = None

    llm_cfg = cfg.get("llm", {})
    provider = str(llm_cfg.get("provider", "vllm")).lower()
    
    DEFAULT_MODEL = None
    client = None
    base_url = ""

    def _try_client(base_url: str, model: str, key: str = "EMPTY"):
        if OpenAI is None:
            return None
        try:
            # 接続確認用の軽量リクエスト
            cli = OpenAI(base_url=base_url, api_key=key or "EMPTY")
            _ = cli.chat.completions.create(
                model=model, 
                messages=[{"role":"user","content":"ping"}], 
                max_tokens=1, 
                temperature=0.0
            )
            return cli
        except Exception as e:
            print(f"⚠️ LLM接続チェック失敗 ({base_url}): {e}")
            return None

    # プロバイダー別の設定読み込み (ハードコードなし、フォールバックなし)
    if provider == "vllm":
        base_url = str(llm_cfg.get("vllm_base_url", ""))
        model = str(llm_cfg.get("vllm_model", ""))
        if base_url and model:
            client = _try_client(base_url, model)
            if client:
                DEFAULT_MODEL = model
        else:
            print("⚠️ vLLMの設定(url/model)が不足しています。")

    elif provider == "ollama":
        base_url = str(llm_cfg.get("ollama_base_url", ""))
        model = str(llm_cfg.get("ollama_model", ""))
        if base_url and model:
            client = _try_client(base_url, model, key="ollama")
            if client:
                DEFAULT_MODEL = model
        else:
            print("⚠️ Ollamaの設定(url/model)が不足しています。")

    else:
        print(f"⚠️ 未対応のLLMプロバイダーです: {provider}")

    LLM_READY = (client is not None) if OpenAI is not None else False
    
    if OpenAI is None:
        print("ℹ️ openai パッケージ未導入: LLM機能は利用できません")
    elif not LLM_READY:
        print(f"❌ LLM接続不可 (provider={provider}, url={base_url})")

    globals()["LLM_CLIENT"] = client
    globals()["LLM_READY"] = LLM_READY
    globals()["LLM_TYPE"]  = provider
    globals()["DEFAULT_MODEL"] = DEFAULT_MODEL
    globals()["VLLM_BASE_URL"] = llm_cfg.get("vllm_base_url")
    globals()["OLLAMA_BASE_URL"] = llm_cfg.get("ollama_base_url")
    
    if LLM_READY:
        print(f"🤖 LLM: type={provider}, ready=True, model={DEFAULT_MODEL}")

    return {
        "DB_CONFIG": DB_CONFIG,
        "GPU_AVAILABLE": GPU_AVAILABLE,
        "LLM_TYPE": provider,
        "LLM_READY": LLM_READY,
        "DEFAULT_MODEL": DEFAULT_MODEL,
    }

# （元セル3の後でも確実に適用するための再適用フック）
if "cfg" in globals() and isinstance(cfg, dict):
    _ = apply_config_to_runtime(cfg)

In [6]:

# === セルX: 偽陰性分析の閾値をConfigから取得 (追記) ===
from typing import Dict, Any
import pandas as pd, numpy as np

def _get_thresholds(cfg: Dict[str, Any]) -> Dict[str, float]:
    a = cfg.get("analysis", {})
    low = float(a.get("fn_threshold_low", 0.2))
    high = float(a.get("fn_threshold_high", 0.7))
    detailed = bool(a.get("enable_detailed_stats", True))
    return {"low": low, "high": high, "detailed": detailed}

def analyze_low_prob_region(false_negatives_df: pd.DataFrame, cfg: Dict[str, Any]) -> Dict[str, Any]:
    th = _get_thresholds(cfg)
    low = th["low"]; high = th["high"]; detailed = th["detailed"]
    if false_negatives_df is None or "prediction_proba" not in false_negatives_df.columns:
        print("⚠️ false_negatives_df 未準備、または prediction_proba 列がありません")
        return {"count": 0, "low_threshold": low, "high_threshold": high}

    df = false_negatives_df
    low_df = df[df["prediction_proba"] < low]
    mid_df = df[(df["prediction_proba"] >= low) & (df["prediction_proba"] < high)]
    high_df = df[df["prediction_proba"] >= high]

    print(f"🎯 低確率領域分析（しきい値 low={low}, high={high}）")
    print(f"  - ML確率 < {low:.2f}: {len(low_df):,}件")
    if detailed:
        print(f"  - {low:.2f} <= ML確率 < {high:.2f}: {len(mid_df):,}件")
        print(f"  - ML確率 >= {high:.2f}: {len(high_df):,}件")

    stats = {
        "low_region_count": int(len(low_df)),
        "mid_region_count": int(len(mid_df)),
        "high_region_count": int(len(high_df)),
        "low_threshold": float(low),
        "high_threshold": float(high),
    }
    if detailed and len(low_df) > 0:
        stats.update({
            "low_mean": float(low_df["prediction_proba"].mean()),
            "low_median": float(low_df["prediction_proba"].median()),
        })
    return stats

# 閾値をグローバルに公開
if "cfg" in globals():
    __th = _get_thresholds(cfg)
    FN_THRESHOLD_LOW = __th["low"]; FN_THRESHOLD_HIGH = __th["high"]; ENABLE_DETAILED_STATS = __th["detailed"]
    globals().update({"FN_THRESHOLD_LOW": FN_THRESHOLD_LOW, "FN_THRESHOLD_HIGH": FN_THRESHOLD_HIGH, "ENABLE_DETAILED_STATS": ENABLE_DETAILED_STATS})
    print(f"🔎 閾値を設定: low={FN_THRESHOLD_LOW}, high={FN_THRESHOLD_HIGH}, detailed={ENABLE_DETAILED_STATS}")


In [7]:

# === 最終セル: Controller API関数 (agent_minimal) (追記) ===
from typing import Tuple, Dict, Any, Optional, List
import os, json, joblib, traceback
from pathlib import Path

# === CHANGELOG ===
# 2025-12-28: artifacts/{session_id}/... 構成に合わせ、brand_keywords と false_negatives の探索パスを更新

def _find_first(*candidates: str) -> Optional[str]:
    for p in candidates:
        if p and os.path.exists(p):
            return p
    return None

def _load_false_negatives(session_id: str):
    import pandas as pd
    # 優先: artifacts/{session_id}/results（現行のフォルダ構成）
    cands = [
        f"artifacts/{session_id}/results/false_negatives_reconstructed.pkl",
        f"artifacts/{session_id}/results/false_negatives.pkl",
        # 互換（旧構成）
        f"results/{session_id}/false_negatives_reconstructed.pkl",
        f"models/{session_id}/false_negatives_reconstructed.pkl",
        f"results/{session_id}/false_negatives.pkl",
    ]
    p = _find_first(*cands)
    if not p:
        return None
    try:
        obj = joblib.load(p)
        if isinstance(obj, pd.DataFrame):
            return obj
        if isinstance(obj, dict):
            if "analysis_df" in obj and isinstance(obj["analysis_df"], pd.DataFrame):
                return obj["analysis_df"]
            if "domains" in obj and "predictions" in obj:
                return pd.DataFrame({
                    "domain": obj["domains"],
                    "prediction_proba": obj["predictions"],
                    "source": obj.get("sources"),
                })
    except Exception:
        return None
    return None


def _normalize_fn_df(fn_df):
    """
    Normalize FN dataframe schema across versions.
    Ensures:
      - "prediction_proba" column exists (float) if any known probability column exists.
      - "domain" column exists if any known domain-like column exists.
    """
    import pandas as pd
    if fn_df is None:
        return None
    if not isinstance(fn_df, pd.DataFrame):
        return fn_df
    df = fn_df.copy()

    # Domain column normalization
    if "domain" not in df.columns:
        for c in ("hostname", "host", "url", "fqdn", "domain_name"):
            if c in df.columns:
                df["domain"] = df[c].astype(str)
                break

    # Probability column normalization
    if "prediction_proba" not in df.columns:
        for c in (
            "ml_probability", "ml_prob", "proba", "pred_proba", "xgb_proba",
            "xgb_probability", "prediction_probability", "score"
        ):
            if c in df.columns:
                df["prediction_proba"] = pd.to_numeric(df[c], errors="coerce")
                break
    # Final fallback
    if "prediction_proba" not in df.columns and "prediction" in df.columns:
        df["prediction_proba"] = pd.to_numeric(df["prediction"], errors="coerce")

    return df


def _load_brand_keywords(session_id: str) -> List[str]:
    # 優先: artifacts/{session_id}/models（現行のフォルダ構成）
    p1 = f"artifacts/{session_id}/models/brand_keywords.json"
    if os.path.exists(p1):
        try:
            return json.loads(Path(p1).read_text(encoding="utf-8"))
        except Exception:
            pass

    # 互換: 旧構成
    p2 = f"models/{session_id}/brand_keywords.json"
    if os.path.exists(p2):
        try:
            return json.loads(Path(p2).read_text(encoding="utf-8"))
        except Exception:
            pass

    # フォールバック: artifacts 配下の最新セッションを探索（ファイルがあるもの）
    try:
        artifacts_dir = Path("artifacts")
        dirs = sorted([d for d in artifacts_dir.glob("*") if d.is_dir()])
        for d in reversed(dirs):
            p = d / "models" / "brand_keywords.json"
            if p.exists():
                return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        pass

    # フォールバック: 旧構成（models/*）
    try:
        models_dir = Path("models")
        dirs = sorted([d for d in models_dir.glob("*") if d.is_dir()])
        for d in reversed(dirs):
            p = d / "brand_keywords.json"
            if p.exists():
                return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        pass

    return []

def _ensure_dirs(base: Path) -> Dict[str, str]:
    results_dir = base / "results"
    handoff_dir = base / "handoff"
    results_dir.mkdir(parents=True, exist_ok=True)
    handoff_dir.mkdir(parents=True, exist_ok=True)
    return {"results_dir": str(results_dir), "handoff_dir": str(handoff_dir)}

def _simple_risk_correction(df, brand_keywords: List[str], low_th: float):
    import pandas as pd
    df = df.copy()
    if "prediction_proba" not in df.columns or "domain" not in df.columns:
        return pd.DataFrame(columns=["domain","prediction_proba","corrected","reason"])
    brands = [b.lower() for b in (brand_keywords or [])]
    def _has_brand(d: str) -> bool:
        d = str(d).lower()
        return any(b and b in d for b in brands[:200])
    def _tld(d: str) -> str:
        parts = str(d).lower().split(".")
        return parts[-1] if len(parts) > 1 else ""
    def _short(d: str) -> bool:
        left = str(d).split(".")[0]
        return len(left) < 10
    corrected, reasons = [], []
    for _, row in df.iterrows():
        ml = float(row.get("prediction_proba", 1.0)); d = str(row.get("domain",""))
        c = False; reason = []
        if ml < low_th:
            if _has_brand(d): c=True; reason.append("brand+lowML")
            if _short(d):     c=True; reason.append("short+lowML")
            tl = _tld(d)
            if tl in ("tk","gq","ml","cf","ga"):
                c=True; reason.append(f"tld({tl})+lowML")
        corrected.append(c); reasons.append(",".join(reason))
    df["corrected"] = corrected; df["reason"] = reasons
    return df

def agent_minimal(session_id: str, inputs: Dict[str, Any] = {"mode":"fn"}, cfg: Dict[str, Any] = None) -> Tuple[str, Dict[str, str]]:
    import pandas as pd
    import matplotlib.pyplot as plt
    from datetime import datetime
    try:
        if not isinstance(session_id, str) or not session_id.strip():
            return "INVALID_INPUT", {"error": "session_idは必須です"}

        local_cfg = load_configuration(cfg_override=(cfg or {}))
        _ = apply_config_to_runtime(local_cfg)
        if local_cfg.get("system", {}).get("cert_only_mode", False):
            print("⚠️ 注意: cert_only_modeですが、ブランド偽装検出は研究の核心機能のため実行します")

        base = Path("artifacts") / session_id
        dirs = _ensure_dirs(base)
        results_dir = Path(dirs["results_dir"]); handoff_dir = Path(dirs["handoff_dir"])

        brand_keywords = _load_brand_keywords(session_id)
        if len(brand_keywords) < 1:
            print("⚠️ ブランドキーワードが見つかりません。基本分析のみ実行します")
        elif len(brand_keywords) < 62:
            print(f"⚠️ ブランドキーワードが{len(brand_keywords)}件 (>=62が推奨)")

        fn_df = _load_false_negatives(session_id)
        fn_df = _normalize_fn_df(fn_df)
        if fn_df is None or fn_df.empty:
            return "NOT_FOUND", {"error": "偽陰性データが見つかりません", "session_id": session_id}

        # Ensure required columns exist
        if "prediction_proba" not in fn_df.columns:
            return "ERROR", {"error": "FNデータに確率列がありません（prediction_proba/ml_probability等）。", "columns": list(fn_df.columns), "session_id": session_id}
        low_th = float(local_cfg.get("analysis", {}).get("fn_threshold_low", 0.2))
        corr_df = _simple_risk_correction(fn_df, brand_keywords, low_th)

        low_df = fn_df[fn_df["prediction_proba"] < low_th]
        corrected_low = corr_df[(corr_df["prediction_proba"] < low_th) & (corr_df["corrected"] == True)]
        improvement_rate = (len(corrected_low) / max(1, len(low_df))) * 100.0

        plt.figure(figsize=(6,4))
        bars = [len(low_df)-len(corrected_low), len(corrected_low)]
        plt.bar(["未補正","補正"], bars)
        plt.title(f"低確率領域(<{low_th:.2f})の補正数 (改善率 {improvement_rate:.1f}%)")
        plt.ylabel("件数")
        out_improve = Path(results_dir) / "fn_detection_improvement.png"
        plt.tight_layout(); plt.savefig(out_improve, dpi=150); plt.close()

        plt.figure(figsize=(6,4))
        fn_df["prediction_proba"].hist(bins=30)
        try:
            plt.axvline(x=low_th, color="red", linestyle="--", label=f"low={low_th}")
            plt.legend()
        except Exception:
            pass
        plt.title("偽陰性の予測確率分布")
        out_dist = Path(results_dir) / "ml_probability_distribution.png"
        plt.tight_layout(); plt.savefig(out_dist, dpi=150); plt.close()

        if "corrected" in corr_df.columns:
            top_reasons = corr_df[corr_df["corrected"]==True]["reason"].value_counts()[:5]
            plt.figure(figsize=(6,4))
            top_reasons.plot(kind="bar")
            plt.title("補正の主因 (上位)")
            out_brand = Path(results_dir) / "brand_detection_results.png"
            plt.tight_layout(); plt.savefig(out_brand, dpi=150); plt.close()
        else:
            out_brand = Path(results_dir) / "brand_detection_results.png"; out_brand.touch()

        report = {
            "session_id": session_id,
            "created_at": datetime.now().isoformat(),
            "metrics": {
                "low_threshold": low_th,
                "low_region_count": int(len(low_df)),
                "corrected_low_count": int(len(corrected_low)),
                "improvement_rate_pct": float(improvement_rate),
            },
            "notes": [
                "LangGraph未使用の最小フォールバック",
                "vLLM→Ollama フォールバックは apply_config_to_runtime で実施",
            ],
        }
        out_json = Path(results_dir) / "agent_performance_report.json"
        out_json.write_text(json.dumps(report, indent=2, ensure_ascii=False), encoding="utf-8")

        out_corr = Path(results_dir) / "false_negatives_corrected.csv"
        corr_df.to_csv(out_corr, index=False, encoding="utf-8")

        handoff_obj = {"false_negatives_df": fn_df, "corrected_df": corr_df, "brand_keywords": brand_keywords, "cfg": local_cfg}
        out_handoff = Path(dirs["handoff_dir"]) / "03_ai_agent_analysis_part1.pkl"
        joblib.dump(handoff_obj, out_handoff)

        Paths = {
            "fn_detection_improvement": str(out_improve),
            "ml_probability_distribution": str(out_dist),
            "brand_detection_results": str(out_brand),
            "agent_performance_report": str(out_json),
            "false_negatives_corrected": str(out_corr),
            "handoff": str(out_handoff),
        }
        return "OK", Paths
    except Exception as e:
        tb = traceback.format_exc(limit=2)
        return "ERROR", {"error": str(e), "traceback": tb}


In [8]:
# === このセルを追加して実行してください（厳格モード） ===
import sys
import _compat.paths as paths

# 1. 実行対象の Session ID を特定
target_sid = None
if 'ai_session_id' in globals() and ai_session_id:
    target_sid = ai_session_id
elif 'session_id' in globals() and session_id:
    target_sid = session_id
else:
    target_sid = paths.RUN_ID 

print(f"🚀 エージェント分析(Part 1)を開始します... (Target Session ID: {target_sid})")

if not target_sid:
    print("❌ Session ID が特定できませんでした。")
    sys.exit(1)

# 2. 関数を実行
# agent_minimal 内部で load_configuration が呼ばれます。
# config.json が無い場合はそこで FileNotFoundError が発生し、停止します。
try:
    # cfg引数をNoneにすることで、load_configurationの自動探索ロジックを利用
    status, result_paths = agent_minimal(session_id=target_sid, cfg=None)

    print(f"\n[Execution Result] Status: {status}")
    if status == "OK":
        print("✅ 処理完了。生成されたファイル:")
        for key, path in result_paths.items():
            print(f" - {key}: {path}")
    else:
        print("❌ エラーが発生しました:\n", result_paths)

except FileNotFoundError as e:
    print(f"\n⛔ 設定エラー: {e}")
except ValueError as e:
    print(f"\n⛔ 設定値エラー: {e}")
except Exception as e:
    import traceback
    print(f"\n❌ 予期せぬエラーが発生しました: {e}")
    print(traceback.format_exc())

🚀 エージェント分析(Part 1)を開始します... (Target Session ID: 2026-01-24_213326)


📖 設定ファイルを読み込みました: config.json
✅ Config loaded successfully (provider=vllm, gpu_auto_detect=True)
🔧 DB設定: {'dbname': 'rapids_data', 'user': 'postgres', 'host': 'localhost', 'port': '5432'} (timeout_s=30)
✅ GPU検出: 利用可能


⚠️ LLM接続チェック失敗 (http://localhost:8000/v1): Connection error.
❌ LLM接続不可 (provider=vllm, url=http://localhost:8000/v1)



[Execution Result] Status: OK
✅ 処理完了。生成されたファイル:
 - fn_detection_improvement: artifacts/2026-01-24_213326/results/fn_detection_improvement.png
 - ml_probability_distribution: artifacts/2026-01-24_213326/results/ml_probability_distribution.png
 - brand_detection_results: artifacts/2026-01-24_213326/results/brand_detection_results.png
 - agent_performance_report: artifacts/2026-01-24_213326/results/agent_performance_report.json
 - false_negatives_corrected: artifacts/2026-01-24_213326/results/false_negatives_corrected.csv
 - handoff: artifacts/2026-01-24_213326/handoff/03_ai_agent_analysis_part1.pkl


In [9]:
import matplotlib.font_manager as fm
fonts = [f.name for f in fm.fontManager.ttflist]
print("Noto Sans CJK JP" in fonts, "IPAexGothic" in fonts, "IPAGothic" in fonts)


True False False
